In [ ]:
import logging

In [ ]:
from atlassian import Bamboo


<br>
That example shows how to clean up Bamboo incomplete or Unknown build results<br>


In [ ]:
logging.basicConfig(level=logging.ERROR)

In [ ]:
REMOVE = True
STATUS_CLEANED_RESULTS = ["Incomplete", "Unknown"]
EXCLUDED_PROJECTS = ["EXCLUDE_PROJECT"]
BAMBOO_LOGIN = "admin"
BAMBOO_PASS = "password"
BAMBOO_URL = "https://bamboo.example.com"

In [ ]:
def get_all_projects():
    return [x["key"] for x in bamboo.projects(max_results=1000)]

In [ ]:
def get_plans_from_project(proj):
    return [x["key"] for x in bamboo.project_plans(proj)]

In [ ]:
def get_branches_from_plan(plan_key):
    return [x["id"] for x in bamboo.search_branches(plan_key, max_results=1000, start=0)]

In [ ]:
def get_results_from_branch(plan_key):
    return [x for x in bamboo.results(plan_key, expand="results.result", max_results=100, include_all_states=True)]

In [ ]:
def remove_build_result(build_key, status):
    result = bamboo.build_result(build_key=build_key)
    if result.get("buildState") == status:
        print("Removing build result - {}".format(build_key))
        if REMOVE:
            bamboo.delete_build_result(build_key=build_key)

In [ ]:
def project_review(plans):
    for plan in plans:
        print("Inspecting {} plan".format(plan))
        branches = get_branches_from_plan(plan)
        for branch in branches:
            build_results = get_results_from_branch(branch)
            for build in build_results:
                build_key = build.get("buildResultKey") or None
                print("Inspecting build - {}".format(build_key))
                if build_key:
                    for status in STATUS_CLEANED_RESULTS:
                        remove_build_result(build_key=build_key, status=status)

In [ ]:
if __name__ == "__main__":
    bamboo = Bamboo(url=BAMBOO_URL, username=BAMBOO_LOGIN, password=BAMBOO_PASS, timeout=180)
    projects = get_all_projects()
    for project in projects:
        if project in EXCLUDED_PROJECTS:
            continue
        print("Inspecting project - {}".format(project))
        results = []
        all_plans_of_project = get_plans_from_project(project)
        project_review(plans=all_plans_of_project)